Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [14]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [35]:
sales = pd.read_csv('sales_train.csv')
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [36]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    # Filter out NaN from item_id before taking unique and converting to int
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].dropna().unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
# Using named aggregation to fix the 'SpecificationError: nested renamer is not supported'
gb = sales.groupby(index_cols,as_index=False).agg(target=('item_cnt_day', 'sum'))

# The previous line to fix column names (gb.columns = ...) is no longer needed
# because named aggregation directly produces the desired column name 'target'
# and does not create multi-level columns when as_index=False is used.

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [37]:
all_data

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


In [38]:
all_data.nunique()

,0
shop_id,60
item_id,21807
date_block_num,34
target,446


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable.

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques.

#### Method 1

In [39]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'] = all_data['item_target_enc'].fillna(0.3343)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Method 2

In [40]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'] = all_data['item_target_enc'].fillna(0.3343)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1]) # you get corr coef matrix where i, i is 1

0.4830386988621699


In [41]:
all_data.iloc[:, :5]

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042
...,...,...,...,...,...
10768834,59,22162,33,0.0,1.556793
10769024,59,22163,33,0.0,0.581395
10769690,59,22164,33,0.0,1.235589
10771216,59,22166,33,0.0,0.295918


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [42]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!**

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection.

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [57]:
import sklearn

# Columns to encode
cols_to_encode = ['item_id']

# 5-fold KFold (no shuffle because time-ordered)
kf = sklearn.model_selection.KFold(n_splits=5, shuffle=False)

# X contains the original columns (clean dataset)
X = all_data[['item_id', 'shop_id', 'date_block_num', 'target']]

# New Output container to store new encoded col: ONLY new columns (not full X copy)
trn_new = pd.DataFrame(index=X.index)

# ----- KFold Target Encoding (Leak-free) -----
for tr_ind, val_ind in kf.split(X):

    X_tr = X.iloc[tr_ind]      # training part for this fold
    X_val = X.iloc[val_ind]    # validation part for this fold

    for col in cols_to_encode:

        # Mean target per category computed ONLY from TRAINING rows
        means = X_tr.groupby(col)['target'].mean()

        # Map validation rows to these means
        enc_val = X_val[col].map(means)

        # Store only the encoded column for the validation rows
        trn_new.loc[val_ind, col + '_mean_target'] = enc_val

# for the only column in question now, fill them with the value as asked
trn_new['item_id_mean_target'] = trn_new['item_id_mean_target'].fillna(0.3323)

# ----- Compute correlation -----
corr = np.corrcoef(X['target'], trn_new['item_id_mean_target'])[0, 1]
print(corr)

grader.submit_tag('KFold_scheme', corr)

0.40923392467105346
Current answer for task KFold_scheme is: 0.40923392467105346


In [52]:
trn_new['item_id_mean_target'].isna().sum()

np.int64(0)

# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time.

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`.

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [55]:
col = 'item_id'

grp_sum = all_data.groupby(col)['target'].transform('sum') # get a col as 'same shape' as col(needed), mapped with their respective item_id's  sum over its group's target vals
grp_count = all_data.groupby(col)['target'].transform('count')

loo_feature = (grp_sum - all_data['target']) / (grp_count - 1)
loo_feature = loo_feature.replace([np.inf, -np.inf], np.nan).fillna(0.3343) # fill missing vals if any with the provided value and replace the infinite vals with NaN (infinite because rare categories have group count eg. 1 then divided by 0)

corr = np.corrcoef(all_data['target'].values, loo_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.4803848311293002
Current answer for task Leave-one-out_scheme is: 0.4803848311293002


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [56]:
col = 'item_id'
alpha = 100
global_mean = 0.3343

grp_mean = all_data.groupby(col)['target'].transform('mean') # means for each item_id
grp_count = all_data.groupby(col)['target'].transform('count') # count for each item_id and map back to the item_ids

smooth_feature = ((grp_mean * grp_count) + (global_mean * alpha)) / (grp_count + alpha)

corr = np.corrcoef(all_data['target'].values, smooth_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797097264
Current answer for task Smoothing_scheme is: 0.4818198797097264


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [58]:
"""
Category	Sales	Cumulative_Sales
A	        10	    10
A	        20	    30
B	        5	    5
B	        10	    15
A	        5	    35
B	        2	    17

cumsum -> sum including curr row(hence subtract current target to make leak free), cumcount -> count before me
cumsum and cumcount are just like transform and they map back to the category they weere grouped by
"""

'\nCategory\tSales\tCumulative_Sales\nA\t        10\t    10\nA\t        20\t    30\nB\t        5\t    5\nB\t        10\t    15\nA\t        5\t    35\nB\t        2\t    17\n\ncumsum and cumcount are just like transform and they map back to the category they weere grouped by\n'

In [60]:
col = 'item_id'

grp_cumsum = all_data.groupby(col)['target'].cumsum() - all_data['target']
grp_cumcnt = all_data.groupby(col)['target'].cumcount()

expmean_feature = grp_cumsum / grp_cumcnt
expmean_feature = expmean_feature.fillna(0.3343)

corr = np.corrcoef(all_data['target'].values, expmean_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211081697
Current answer for task Expanding_mean_scheme is: 0.5025245211081697


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [62]:
STUDENT_EMAIL = ...
STUDENT_TOKEN = ...
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.40923392467105346
Task Leave-one-out_scheme: 0.4803848311293002
Task Smoothing_scheme: 0.4818198797097264
Task Expanding_mean_scheme: 0.5025245211081697
